In [1]:
import pandas as pd

df = pd.read_csv("../data/processed/churn_predictions.csv")
df.head()


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment,Cluster,PCA1,PCA2,Churn,Churn_Probability
0,12346.0,326,1,77183.60,1,1,5,115,Lost Customers,3,4.106623,5.433579,1,0.950000
1,12347.0,2,7,4310.00,5,5,5,555,Champions,0,0.742411,-0.671300,0,NaN
2,12348.0,75,4,1797.24,2,4,4,244,At Risk,0,0.024802,-0.174954,0,0.001117
3,12349.0,19,1,1757.55,4,1,4,414,Potential Loyalists,0,-0.028036,-0.735124,0,NaN
4,12350.0,310,1,334.40,1,1,2,112,Lost Customers,1,-1.235466,1.834946,1,NaN


In [2]:
def churn_risk_bucket(prob):
    if prob >= 0.7:
        return "High Risk"
    elif prob >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

df["Churn_Risk"] = df["Churn_Probability"].apply(churn_risk_bucket)


In [3]:
def retention_strategy(row):
    if row["Segment"] == "Champions" and row["Churn_Risk"] == "High Risk":
        return "Personal outreach + premium offer"
    
    elif row["Segment"] == "Loyal Customers" and row["Churn_Risk"] == "Medium Risk":
        return "Loyalty points incentive"
    
    elif row["Segment"] == "At Risk" and row["Churn_Risk"] == "High Risk":
        return "Discount coupon"
    
    elif row["Segment"] == "Potential Loyalists":
        return "Personalized email campaign"
    
    elif row["Segment"] == "Lost Customers":
        return "Re-engagement campaign"
    
    else:
        return "No immediate action"


In [4]:
df["Retention_Action"] = df.apply(retention_strategy, axis=1)


In [5]:
df["Retention_Action"].value_counts()


Retention_Action
No immediate action            3228
Re-engagement campaign          664
Personalized email campaign     319
Discount coupon                 127
Name: count, dtype: int64

In [6]:
pd.crosstab(df["Churn_Risk"], df["Retention_Action"])


Retention_Action,Discount coupon,No immediate action,Personalized email campaign,Re-engagement campaign
Churn_Risk,,,,
High Risk,127,68,0,167
Low Risk,0,3160,319,497


In [7]:
df.to_csv("../data/processed/customer_retention_actions.csv", index=False)


In [8]:
import pandas as pd

df = pd.read_csv("../data/processed/customer_retention_actions.csv")
df.head()


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment,Cluster,PCA1,PCA2,Churn,Churn_Probability,Churn_Risk,Retention_Action
0,12346.0,326,1,77183.60,1,1,5,115,Lost Customers,3,4.106623,5.433579,1,0.950000,High Risk,Re-engagement campaign
1,12347.0,2,7,4310.00,5,5,5,555,Champions,0,0.742411,-0.671300,0,NaN,Low Risk,No immediate action
2,12348.0,75,4,1797.24,2,4,4,244,At Risk,0,0.024802,-0.174954,0,0.001117,Low Risk,No immediate action
3,12349.0,19,1,1757.55,4,1,4,414,Potential Loyalists,0,-0.028036,-0.735124,0,NaN,Low Risk,Personalized email campaign
4,12350.0,310,1,334.40,1,1,2,112,Lost Customers,1,-1.235466,1.834946,1,NaN,Low Risk,Re-engagement campaign


In [9]:
ACTION_COSTS = {
    "Personal outreach + premium offer": 50,
    "Loyalty points incentive": 20,
    "Discount coupon": 30,
    "Personalized email campaign": 5,
    "Re-engagement campaign": 10,
    "No immediate action": 0
}


In [11]:
df["Expected_Revenue"] = df["Monetary"] * (1 - df["Churn_Probability"])


In [12]:
df["Retention_Cost"] = df["Retention_Action"].map(ACTION_COSTS)


In [13]:
df["ROI"] = df["Expected_Revenue"] - df["Retention_Cost"]


In [14]:
df["Final_Action"] = df.apply(
    lambda x: x["Retention_Action"]
    if (x["Churn_Risk"] == "High Risk" and x["ROI"] > 0)
    else "No Action",
    axis=1
)


In [15]:
df["ROI"].sum()


np.float64(2187584.9324420756)

In [16]:
df.groupby("Final_Action")["ROI"].sum().sort_values(ascending=False)


Final_Action
No Action                 2.181115e+06
Re-engagement campaign    3.849180e+03
Discount coupon           2.482484e+03
No immediate action       1.380373e+02
Name: ROI, dtype: float64

In [17]:
df["Final_Action"].value_counts()


Final_Action
No Action                 4293
No immediate action         39
Discount coupon              5
Re-engagement campaign       1
Name: count, dtype: int64

In [18]:
df.to_csv("../data/processed/roi_optimized_retention.csv", index=False)
